In [14]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os
import time
import numpy as np

receipt_data_file = glob.glob('C:/Users/again/Desktop/shinhancard/shinhancard_*.html') # 지정 폴더안에 있는 모든 파일을 불러내기 위함

total_df = []           # 일별 DataFrame 모음
total_df_region = []    # 지역별 DataFrame 모음
sheet_nm = []           # Excel Sheet Name 모음

# 위의 'total_df'와 'sheet_nm'에 데이터를 삽입하기 위한 for 구문
for url in receipt_data_file :
    driver = webdriver.Chrome('c:/chromedriver.exe')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    title_tag = soup.select('head > title')
    title = title_tag[0].text

    # 암호된 문서일 경우 if 구문으로 들어감
    if title == '' :
        pw = '950730'
        driver.find_element_by_id('password').send_keys(pw)
        driver.find_element_by_xpath('//*[@id="aform"]/span/input').click()

    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    # 암호된 문서가 아닐 경우 아래있는 코드로 바로 들어감
    tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
    tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

    for n in range(len(tables_middleSum )) :
        tables.remove(tables_middleSum[n])
    
    # 명세서에서 추출한 데이터 string으로 변환하여 분리
    seperate_tables = []                          
    for n in range(len(tables)) :
        table = str(tables[n].text).split('\n')
        seperate_tables = seperate_tables + table

    # seperate_tables에서 '사용날짜' 추출   
    date_list = []
    for n in range(1,len(seperate_tables), 12) :
        date = seperate_tables[n]
        date_list.append(date)

    # seperate_tables에서 '사용처' 데이터만 추출    
    store_list = []
    for n in range(3,len(seperate_tables),12) :
        store = seperate_tables[n].replace(" ", "")
        store_list.append(store)

    # seperate_tables에서 추출된 '사용처' 데이터 --> 카카오 지도 API --> '사용처'에 대한 주소 추출      
    location_list = []
    def create_url(search_text):
        headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}
        base = "https://dapi.kakao.com/v2/local/search/keyword.json"
        param_query = "?query=" +search_text
    #     print(f'base + param_query: {base + param_query}')
        respond = requests.get(base + param_query,headers=headers)
        return respond.json()

    for n in range(len(store_list)) :
        store_name = store_list[n]
        result = create_url(store_name)
        tmp1 = result['documents']
        try :
            tmp2 = tmp1[0]
            tmp3 = tmp2['address_name']
            location_list.append(tmp3)
        except :
            tmp2 = np.nan
            location_list.append(tmp2)

    # seperate_tables에서 '사용금액' 데이터만 추출  
    price_list = [] 
    for n in range(4,len(seperate_tables),12) :
        price = seperate_tables[n]
        price_replace = int(price.replace(',',''))
        price_list.append(price_replace)
    
    # 추출한 데이터로 DataFrame 변환
    table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }
    
    df = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])
    df_sum = pd.DataFrame(df.sum(), columns=['합계'])
    df_total = df.append(df_sum.T)
    df_total.loc['합계', '사용날짜'] = '전체'
    df_total.loc['합계', '사용처'] = '전체'
    df_total.loc['합계', '사용처주소'] = '전체'
    total_df.append(df_total)
    
    ############################################################################# 여기서부터 지역별 데이터 파일데이터 추출
    
    df_dis_nan = df.dropna()

    df_address_list = list(df_dis_nan['사용처주소'])
    df_region_list = [ ' '.join(address.split()[:2]) for address in df_address_list]

    df_price_list = list(df_dis_nan['사용금액'])

    table_data_1 = {'사용지역':df_region_list, '사용금액':df_price_list}
    col_list = ['사용지역', '사용금액']
    df_Dup = pd.DataFrame(table_data_1 , columns = col_list ) # 중복값이 있는 df
    df_NoDup = df_Dup.drop_duplicates(subset = '사용지역')  # 중복값이 없는 df

    index_name_1 = df_Dup['사용지역']  # 중복값이 있는 사용지역 series
    df_Dup_name = df_Dup.rename(index_name_1)
    df_Dup_name = df_Dup_name.drop(['사용지역'], axis = 1)  # 중복값이 있고 사용지역이 index인 df

    index_name_2 = df_NoDup['사용지역']  # 중복값이 없는 사용지역 series
    df_NoDup_name = df_NoDup.rename(index_name_2)
    df_NoDup_name = df_NoDup_name.drop(['사용지역'], axis = 1)  # 중복값이 없고 사용지역이 index인 df

    for nm in list(index_name_2) :
        price_data = df_Dup_name.loc[nm]['사용금액']
        price_sum = price_data.sum()
        df_NoDup_name.loc[nm]['사용금액'] = price_sum

    name_region = df_Dup['사용지역']
    visit_counter = dict(name_region.value_counts())
    counter_list = []
    for k in index_name_2 :
        counter_list.append(visit_counter[k])
    
    df_NoDup_name.insert(0, '방문횟수', counter_list)
    total_df_region.append(df_NoDup_name)
    
    ############################################################################# 여기까지 지역별 데이터 파일데이터 추출  
    
    tables_date = soup.select('body > div.wrap.check > div.content > div.receiveDate > span.date')
    nm = str(tables_date[0].text)
    sheet_nm.append(nm)
    
    driver.close()

# 모아진 DataFrame을 Excel로변환
folder = 'C:/Users/again/Desktop/Card'

if not os.path.exists(folder) :
    os.makedirs(folder)
    
excel_writer = pd.ExcelWriter('C:/Users/again/Desktop/Card/CardUseList.xlsx', engine='xlsxwriter')
ToExcel_data =[]

for n in range(len(total_df)) :
    ToAppend_total_df = total_df[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'일별 데이터')
    ToExcel_data.append(ToAppend_total_df)
    ToAppend_total_region_df = total_df_region[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'지역별 데이터')
    ToExcel_data.append(ToAppend_total_region_df)

# for n in range(len(total_df)) :
#     ToAppend_df = total_df[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'일별 데이터')
#     ToExcel_data.append(ToAppend_df)
        
# for n in range(len(total_df_region)) :
#     ToAppend_df = total_df_region[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n]+'지역별 데이터')
#     ToExcel_data.append(ToAppend_df)
        
excel_writer.save()